In [15]:
import os
import csv
from itertools import combinations

# Configuration
GOPHERSAT_PATH = r"gophersat.exe"  # chemin vers le solveur
SCHEDULES_IMP_DIR = r"schedules_imp"      # Dossier des emplois du temps
DURATION = 15               # Durée en minutes (15, 30, 45, 60)
IMPORTANCE = 5
 

In [16]:
def lire_emplois_du_temps(dossier):
    """
    Lit tous les fichiers CSV dans le dossier et retourne :
      - emplois : un dictionnaire où chaque clé correspond à un participant et
                  la valeur est un dictionnaire {jour: {horaire: statut}}
                  Le statut est un entier de 0 à 5.
                  0 indique que le participant est disponible,
                  1 à 4 représentent des réunions avec des niveaux d'importance croissante,
                  5 représente une réunion extrêmement importante.
      - jours : la liste des jours considérés
      - horaires : la liste des horaires (extrait du header des CSV)
    """
    fichiers = [f for f in os.listdir(dossier) if f.endswith(".csv")]
    if not fichiers:
        raise FileNotFoundError("Aucun fichier CSV trouvé dans le dossier")

    jours = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    emplois = {}  # clé : identifiant du participant, valeur : emploi du temps
    horaires = None

    for fichier in fichiers:
        with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
            lecteur = csv.reader(f)
            header = next(lecteur)
            # La première colonne est "Jour", le reste sont les horaires
            if horaires is None:
                horaires = header[1:]
            emploi = {}  # emploi du temps pour ce participant
            for ligne in lecteur:
                jour = ligne[0]
                if jour not in jours:
                    continue
                # Pour chaque jour, créer un dictionnaire {horaire: statut}
                emploi[jour] = {}
                for i, statut in enumerate(ligne[1:]):
                    # Convertir le statut en entier:
                    # 0 : disponible,
                    # 1 à 4 : réunion avec des niveaux d'importance croissante,
                    # 5 : réunion extrêmement importante.
                    importance = int(statut)
                    if not 0 <= importance <= 5:
                        raise ValueError(
                            f"Valeur de statut invalide {importance} dans le fichier {fichier} "
                            f"pour le jour {jour} et l'horaire {horaires[i]}"
                        )
                    emploi[jour][horaires[i]] = importance
            # On peut utiliser le nom du fichier comme identifiant du participant
            emplois[fichier] = emploi

    return emplois, jours, horaires

In [17]:

def generer_fichier_cnf_complet(emplois, jours, horaires, duree, importance):
    """
    Génère un fichier CNF (au format DIMACS) pour la planification d'une réunion d'importance donnée.

    Modélisation :
      - Pour chaque participant p, jour d et horaire s, on définit une variable X_{p,d,s} indiquant
        que le participant peut accueillir une réunion d'importance 'importance' à ce créneau.
      - Pour chaque candidat de réunion (début de bloc) Y_{d,s}, pour s tel que le bloc de longueur m 
        (duree//15) tient dans la liste des horaires, on définit une variable Y_{d,s}.

    Contraintes :
      1. Exactement un bloc de réunion est choisi (variables Y) :
           - Au moins un bloc choisi.
           - Pas deux blocs simultanément (exclusion mutuelle).
      2. Lien entre Y et X :
           Pour chaque candidat Y_{d,s} et pour chaque participant p, et pour chaque créneau du bloc
           (s, s+1, ..., s+m-1), on impose :
             Y_{d,s} -> X_{p,d,t}   (en CNF : -Y_{d,s} v X_{p,d,t})
      3. Disponibilité des participants :
           Pour chaque participant p, jour d, horaire s, si le participant a déjà un engagement 
           d'une importance supérieure ou égale à l'importance de la réunion à planifier 
           (i.e. emploi[p][d][s] >= importance), alors X_{p,d,s} est forcé à faux.
      4. Interdiction des blocs traversant la pause de midi.
    """
    m = duree // 15  # Nombre de créneaux consécutifs requis

    clauses = []
    next_var = 1

    # 1. Création des variables X : X_vars[(participant, jour, horaire)]
    X_vars = {}
    for participant in emplois:
        for d in jours:
            for t in horaires:
                X_vars[(participant, d, t)] = next_var
                next_var += 1

    # 2. Création des variables Y : Y_vars[(jour, horaire_debut)]
    Y_vars = {}
    for d in jours:
        # On ne considère que les candidats qui permettent d'avoir m créneaux consécutifs
        for i in range(len(horaires) - m + 1):
            t_debut = horaires[i]
            Y_vars[(d, t_debut)] = next_var
            next_var += 1

    # ======================
    # Contrainte (1) : Exactement un bloc de réunion est choisi (variables Y)
    # a) Au moins un bloc choisi :
    clause = " ".join(str(Y_vars[(d, t)]) for (d, t) in Y_vars) + " 0"
    clauses.append(clause)
    # b) Au plus un bloc choisi : pour chaque paire distincte, on empêche la double sélection
    for (d1, t1), (d2, t2) in combinations(Y_vars.keys(), 2):
        clauses.append(f"-{Y_vars[(d1, t1)]} -{Y_vars[(d2, t2)]} 0")

    # ======================
    # Contrainte (2) : Lien entre Y et X
    # Si le bloc Y_{d,t_debut} est choisi, alors pour chaque participant et pour chaque créneau
    # du bloc, le participant doit être disponible (X_{p,d,t} doit être vrai)
    for (d, t_debut) in Y_vars:
        start_index = horaires.index(t_debut)
        for participant in emplois:
            for j in range(m):
                t = horaires[start_index + j]
                # Clause: -Y_{d,t_debut} v X_{p,d,t}
                clauses.append(f"-{Y_vars[(d, t_debut)]} {X_vars[(participant, d, t)]} 0")

    # ======================
    # Contrainte (3) : Disponibilité des participants pour une réunion d'importance 'importance'
    # Un participant n'est disponible à un créneau que si son emploi du temps indique
    # un engagement d'une importance strictement inférieure à 'importance'.
    for participant, emploi in emplois.items():
        for d in jours:
            for t in horaires:
                if emploi[d][t] >= importance:
                    # Forcer X_{p,d,t} à faux si le participant est engagé avec un niveau >= importance.
                    clauses.append(f"-{X_vars[(participant, d, t)]} 0")
                    
    # ======================
    # Contrainte (4) : Interdiction des blocs traversant la pause de midi
    for (d, t_debut) in Y_vars:
        start_index = horaires.index(t_debut)
        for j in range(m):
            t = horaires[start_index + j]
            if horaires[start_index] < "12:00" and t >= "14:00":
                clauses.append(f"-{Y_vars[(d, t_debut)]} 0")
                break  # Une fois la contrainte appliquée, on passe au candidat suivant

    # Écriture du fichier CNF
    total_vars = next_var - 1
    total_clauses = len(clauses)
    with open("reunion.cnf", "w") as f:
        f.write(f"p cnf {total_vars} {total_clauses}\n")
        for clause in clauses:
            f.write(clause + "\n")

    print(f"Fichier CNF généré avec {total_vars} variables et {total_clauses} clauses.")
    return True, Y_vars


In [18]:

def resoudre_toutes_les_solutions(Y_vars):
    """
    Résout le problème en énumérant toutes les solutions (selon la CNF générée)
    et retourne toutes les solutions ayant le coût minimal.
   
    Le coût d'un créneau est défini comme la somme, pour tous les participants,
    de l'importance actuelle des réunions dans les créneaux utilisés pour la nouvelle réunion.
    (0 signifie libre, 1 à 4 ou 5 indiquent des réunions de niveau croissant.)
   
    La fonction retourne toutes les solutions ayant le coût minimal.
    """
    solutions = []  # Chaque élément sera (solution, cost, (jour, t_debut))
    m = DURATION // 15  # Nombre de créneaux consécutifs requis pour la réunion
    
    while True:
        # Appelle le SAT solver et redirige la sortie dans "resultat.txt"
        os.system(f"{GOPHERSAT_PATH} reunion.cnf > resultat.txt")
        solution = None
        
        with open("resultat.txt", "r") as f:
            for ligne in f:
                if ligne.startswith("v "):
                    solution = list(map(int, ligne.strip().split()[1:-1]))
                    
        if not solution:
            print("✅ Plus aucune solution trouvée, arrêt de l'exploration.")
            break
            
        # Identifier les variables Y activées dans la solution
        reunion_cree_vars = [var for var in solution if var > 0 and var in Y_vars.values()]
        
        if reunion_cree_vars:
            # En principe, la contrainte "exactement un bloc" garantit qu'il y en a un
            for (d, t_debut), var in Y_vars.items():
                if var in reunion_cree_vars:
                    try:
                        start_index = horaires.index(t_debut)
                    except ValueError:
                        print(f"⚠️ Erreur: horaire {t_debut} non trouvé dans la liste d'horaires.")
                        continue
                        
                    # Calcul du coût pour le bloc commençant à t_debut le jour d
                    block_cost = 0
                    for participant in emplois:
                        for j in range(m):
                            t = horaires[start_index + j]
                            block_cost += emplois[participant][d][t]
                            
                    solutions.append((solution, block_cost, (d, t_debut)))
        else:
            print("❌ Aucun créneau de réunion trouvé dans cette solution.")
            
        # Exclure cette solution pour forcer le solver à explorer une solution différente
        with open("reunion.cnf", "a") as f:
            f.write(" ".join(f"-{var}" for var in reunion_cree_vars) + " 0\n")
    
    # Trouver toutes les solutions avec le coût minimal
    if solutions:
        min_cost = min(solution[1] for solution in solutions)
        best_solutions = [sol for sol in solutions if sol[1] == min_cost]
                
        return best_solutions
    else:

        return None

In [19]:
def afficher_resultats_detailles(solutions, Y_vars, emplois, jours, horaires, duree):
    """
    Affiche les résultats détaillés pour toutes les solutions trouvées.
    
    :param solutions: Une liste de tuples (solution, cost, (jour, horaire)).
    :param Y_vars: Dictionnaire des variables Y du modèle CNF.
    :param emplois: Dictionnaire des emplois du temps des participants.
    :param jours: Liste des jours considérés.
    :param horaires: Liste des horaires extraits du header des CSV.
    :param duree: Durée de la réunion (en minutes).
    """
    print(f"=== Créneaux possibles pour une réunion de {duree} minutes ===\n")
    
    # Itérer sur toutes les solutions trouvées
    for idx, (solution, cost, (jour, horaire)) in enumerate(solutions, start=1):
        m = duree // 15  # Nombre de créneaux nécessaires
        try:
            start_index = horaires.index(horaire)
        except ValueError:
            print(f"⚠️ Erreur: l'horaire {horaire} n'est pas dans la liste d'horaires.")
            continue
        
        # Calculer l'horaire de fin en utilisant l'index dans la liste des horaires
        if start_index + m < len(horaires):
            end_time = horaires[start_index + m]
        else:
            end_time = "??"
        
        print(f"Option {idx}: {jour} {horaire}-{end_time}")
        if cost == 0:
            print("✅ Tous les participants sont disponibles")
        else:
            print(f"⚠️ Coût: {cost}")
        print()  # Ligne vide pour séparer les options



In [20]:
emplois, jours, horaires = lire_emplois_du_temps(SCHEDULES_IMP_DIR)
print(f"Créneaux occupés chargés")
success, Y_vars = generer_fichier_cnf_complet(emplois, jours, horaires, DURATION,IMPORTANCE)
resultat = resoudre_toutes_les_solutions(Y_vars)
if resultat:
    afficher_resultats_detailles(resultat, Y_vars, emplois, jours, horaires, DURATION)
else:
    print("🚫 Aucune solution possible.")




FileNotFoundError: [WinError 3] The system cannot find the path specified: 'schedules_imp'